In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
import pandas as pd

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

train_copy=train.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']



scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # (batch, 1, features)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)


In [ ]:


class Tabular1DCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super(Tabular1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool(x).squeeze(-1)  # shape: (batch_size, channels)
        x = self.fc(x)
        return x


model = Tabular1DCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}")


model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_classes = torch.argmax(predictions, dim=1)
    print(classification_report(y_test_tensor, predicted_classes))

Epoch 1/20 - Loss: 0.7157
Epoch 2/20 - Loss: 0.7117
Epoch 3/20 - Loss: 0.7080
Epoch 4/20 - Loss: 0.7047
Epoch 5/20 - Loss: 0.7016
Epoch 6/20 - Loss: 0.6987
Epoch 7/20 - Loss: 0.6961
Epoch 8/20 - Loss: 0.6936
Epoch 9/20 - Loss: 0.6914
Epoch 10/20 - Loss: 0.6892
Epoch 11/20 - Loss: 0.6873
Epoch 12/20 - Loss: 0.6854
Epoch 13/20 - Loss: 0.6837
Epoch 14/20 - Loss: 0.6821
Epoch 15/20 - Loss: 0.6806
Epoch 16/20 - Loss: 0.6793
Epoch 17/20 - Loss: 0.6780
Epoch 18/20 - Loss: 0.6768
Epoch 19/20 - Loss: 0.6757
Epoch 20/20 - Loss: 0.6746
              precision    recall  f1-score   support

           0       0.56      1.00      0.72     17783
           1       0.00      0.00      0.00     14069

    accuracy                           0.56     31852
   macro avg       0.28      0.50      0.36     31852
weighted avg       0.31      0.56      0.40     31852



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
import numpy as np


X = X.values if hasattr(X, 'values') else X  
y = y.values if hasattr(y, 'values') else y


class Tabular1DCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super(Tabular1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return x


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

all_preds = []
all_labels = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\nFold {fold+1}")
    
   
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

  
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

   
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1)
    X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).unsqueeze(1)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

   
    model = Tabular1DCNN(in_channels=1, num_classes=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

  
    for epoch in range(20):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/20 - Loss: {loss.item():.4f}")

   
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_preds = torch.argmax(val_outputs, dim=1)

    all_preds.extend(val_preds.cpu().numpy())
    all_labels.extend(y_val_tensor.cpu().numpy())


print("Cross-Validation Classification Report ")
print(classification_report(all_labels, all_preds))